In [2]:
!pip install -q transformers datasets accelerate sentencepiece huggingface_hub nlpaug

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.5/410.5 kB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 96.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 58.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
from datasets import load_dataset, concatenate_datasets
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
import torch
from torch.nn import BCEWithLogitsLoss
import torch.nn as nn
from sklearn.metrics import accuracy_score, f1_score
from transformers import DataCollatorWithPadding

In [4]:
ds = load_dataset("Silly-Machine/TuPyE-Dataset", "multilabel")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/8.28k [00:00<?, ?B/s]

multilabel/multilabel_train.csv:   0%|          | 0.00/4.79M [00:00<?, ?B/s]

multilabel/multilabel_test.csv:   0%|          | 0.00/1.18M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/34934 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/8734 [00:00<?, ? examples/s]

In [192]:
print("colunas do dataset:", ds["train"].column_names)

colunas do dataset: ['text', 'aggressive', 'hate', 'ageism', 'aporophobia', 'body_shame', 'capacitism', 'lgbtphobia', 'political', 'racism', 'religious_intolerance', 'misogyny', 'xenophobia', 'other', 'researcher', 'year', 'source']


In [6]:
ds_split = ds['train'].train_test_split(test_size=0.2)

In [7]:
model_name = 'neuralmind/bert-base-portuguese-cased'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)
model.config.hidden_dropout_prob = 0.3
model.config.attention_probs_dropout_prob = 0.3
model.config.problem_type = "multi_label_classification"

tokenizer_config.json:   0%|          | 0.00/43.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/647 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/210k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at neuralmind/bert-base-portuguese-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [10]:
# @title
def tokenize_function(examples):
    return tokenizer(
        examples["text"],
        padding="max_length",
        truncation=True,
        max_length=512,
    )

# Tokenizar os dados
tokens_ds = ds_split.map(tokenize_function, batched=True)

def combine_labels(examples):
  label_keys= ['aggressive', 'hate', 'misogyny']
  labels = [examples[key] for key in label_keys]

  return {"labels": labels}
tokens_ds = tokens_ds.map(combine_labels, batched=False)


Map:   0%|          | 0/27947 [00:00<?, ? examples/s]

Map:   0%|          | 0/6987 [00:00<?, ? examples/s]

Map:   0%|          | 0/27947 [00:00<?, ? examples/s]

Map:   0%|          | 0/6987 [00:00<?, ? examples/s]

In [11]:
columns_to_remove = [col for col in tokens_ds['train'].column_names if col not in ['input_ids', 'attention_mask', 'labels']]
tokens_ds = tokens_ds.remove_columns(columns_to_remove)

In [12]:
class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
      #print("dados recebidos:", inputs)
      labels = inputs.pop("labels")

      if labels is None:
          raise KeyError("Labels not found in inputs")


      labels = torch.tensor(labels).float().to(model.device)


      outputs = model(**inputs)
      logits = outputs.logits

      loss_fct = BCEWithLogitsLoss()
      loss = loss_fct(logits, labels)

      return (loss, outputs) if return_outputs else loss


In [13]:
def compute_metrics(p):
    preds = torch.sigmoid(torch.tensor(p.predictions)).numpy()
    preds = (preds > 0.5).astype(int)
    labels = p.label_ids

    return {
        "accuracy": accuracy_score(labels, preds),
        "f1_micro": f1_score(labels, preds, average="micro"),
        "f1_macro": f1_score(labels, preds, average="macro"),
    }

In [14]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=5,
    per_device_train_batch_size=16,
    eval_strategy="epoch",
    save_strategy="epoch",
    label_names=["labels"],
    logging_steps=100,
    learning_rate=2e-5,
    weight_decay=0.01,
    gradient_accumulation_steps=2,
    remove_unused_columns=False,
    dataloader_num_workers=2,

)

trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=tokens_ds['train'],
    eval_dataset=tokens_ds['test'],
    compute_metrics=compute_metrics,
    data_collator=data_collator,


)

In [201]:
print("Colunas disponíveis no dataset:", tokens_ds["train"].column_names)
print("Exemplo do dataset:", tokens_ds["train"][0])

Colunas disponíveis no dataset: ['input_ids', 'attention_mask', 'labels']
Exemplo do dataset: {'input_ids': [101, 538, 3539, 128, 173, 160, 9840, 1354, 22290, 268, 106, 108, 781, 244, 324, 15101, 12126, 22280, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [1]:
trainer.train()

NameError: name 'trainer' is not defined

In [1]:
from sklearn.metrics import confusion_matrix

# Get predictions using the trainer object
predictions = trainer.predict(ds_split['test'])
# Access predicted labels and ground truth labels
y_pred = predictions.predictions.argmax(-1)
y_test = predictions.label_ids

# Compute and print the confusion matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)


NameError: name 'trainer' is not defined